<a href="https://colab.research.google.com/github/chloesung/Econometrics/blob/main/2.%20Instrument%20Variables%20Approach/Wooldridge/Wooldridge_Ch15(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
pip install wooldridge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.1 MB 8.7 MB/s 


In [2]:
pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 8.4 MB/s 
     |████████████████████████████████| 76 kB 6.7 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.0 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
spacy 3.4.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import re
import math

import seaborn as sns
sns.set_palette("pastel")
sns.set(style="whitegrid")
import warnings
import matplotlib.pyplot as plt
pal = sns.color_palette('Set1')

from scipy.stats import t
from statsmodels.formula.api import ols
from statsmodels.api import add_constant
pd.options.display.float_format = '{:.5f}'.format
plt.rcParams['axes.unicode_minus'] = False
from linearmodels import IV2SLS
import wooldridge

warnings.filterwarnings(action='ignore')
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina' #화질 좋게 해주기")

# 1. Effect of Education on Fertility

## a) EDA

In [18]:
# Data Import
df = wooldridge.data('fertil1')[['kids','educ','age','agesq','black','east','northcen','west','farm',
                                 'othrural','town','smcity','y74','y76','y78','y80','y82','y84','meduc','feduc']]
df

,kids,educ,age,agesq,black,east,northcen,west,farm,othrural,town,smcity,y74,y76,y78,y80,y82,y84,meduc,feduc
0,4,12,48,2304,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8,8
1,3,17,46,2116,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8,18
2,2,12,53,2809,0,0,1,0,0,1,0,0,0,0,0,0,0,0,7,8
3,2,12,42,1764,0,0,1,0,0,0,1,0,0,0,0,0,0,0,12,10
4,2,12,51,2601,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124,1,12,41,1681,0,0,0,1,0,0,1,0,0,0,0,0,0,1,12,12
1125,0,19,42,1764,0,0,0,1,0,0,1,0,0,0,0,0,0,1,10,15
1126,3,15,35,1225,0,0,0,1,0,0,1,0,0,0,0,0,0,1,12,10
1127,2,12,46,2116,0,0,1,0,1,0,0,0,0,0,0,0,0,1,12,12


- ***educ***: endogenous variable
- ***meduc***, ***feduc***: instrument for *educ*

In [24]:
# Testing for Endogeneity

# Step 1
res = ols('educ ~ age + agesq + black + east + northcen + west + farm + othrural + town + smcity + y74 + y76 + y78 + y80 + y82 + y84 + meduc + feduc', data = df).fit()
resid = res.resid

# Step 2
res = ols('kids ~ educ + age + agesq + black + east + northcen + west + farm + othrural + town + smcity + y74 + y76 + y78 + y80 + y82 + y84 + resid', data = df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   kids   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     9.206
Date:                Mon, 05 Sep 2022   Prob (F-statistic):           6.17e-24
Time:                        18:33:10   Log-Likelihood:                -2091.0
No. Observations:                1129   AIC:                             4220.
Df Residuals:                    1110   BIC:                             4316.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.2412      3.135     -2.310      0.0

- $p$ value = .483
- $t$ value = .702
- Thus, ***educ*** is indeed endogenous

## b) OLS


In [19]:
# Regression
res = ols('kids ~ educ + age + agesq + black + east + northcen + west + farm + othrural + town + smcity + y74 + y76 + y78 + y80 + y82 + y84', data = df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   kids   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     9.723
Date:                Mon, 05 Sep 2022   Prob (F-statistic):           2.42e-24
Time:                        18:18:39   Log-Likelihood:                -2091.2
No. Observations:                1129   AIC:                             4218.
Df Residuals:                    1111   BIC:                             4309.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -7.7425      3.052     -2.537      0.0

- $\beta_educ = -.153$ 
- $se = .018$

## c) IV

In [20]:
df = add_constant(df, has_constant="add")
dep = df['kids']
exog = df[['const','age','agesq','black','east','northcen','west','farm', 'othrural','town','smcity','y74','y76','y78','y80','y82','y84']]
endog = df['educ']
instr = df[['meduc','feduc']]

resultIV = IV2SLS(dep, exog, endog, instr).fit() 

print(resultIV.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   kids   R-squared:                      0.1281
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1148
No. Observations:                1129   F-statistic:                    150.13
Date:                Mon, Sep 05 2022   P-value (F-stat)                0.0000
Time:                        18:18:42   Distribution:                 chi2(17)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -7.2412     3.1890    -2.2707     0.0232     -13.491     -0.9910
age            0.5236     0.1395     3.7540     0.00

- $\beta_educ = -.153$ 
- $se = .041$
- The OLS and 2SLS estimates of $\beta_{educ}$ are *not statistically* different.
- Therefore, in this case, we conclude that the difference between 2SLS and OLS could be entirely due to sampling error.